# Temporal Graphs and Path Data

## Motivation and Learning Objectives

In this tutorial we will introduce the representation of temporal graph data in the `Temporal Graph` class and how such data can be used to calculate time respecting paths.


In [3]:
import torch
from torch_geometric.data import TemporalData
from torch_geometric.utils import remove_isolated_nodes
import numpy as np
import pathpyG as pp

pp.config['torch']['device'] = 'cpu'

In [4]:
tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.N)
print(t.M)

9
20


In [5]:
print(t.data)

TemporalData(src=[20], dst=[20], t=[20])


In [6]:
td = TemporalData(
    src = torch.Tensor([0,1,2,0]),
    dst = torch.Tensor([1,2,3,1]), 
    t = torch.Tensor([0,1,2,3]))
print(td)

TemporalData(src=[4], dst=[4], t=[4])


In [7]:
t2 = pp.TemporalGraph(td)

In [8]:
t1 = t.get_window(0,4)
print(t1)
print(t1.N)
print(t1.M)

Temporal Graph with 5 nodes, 4 unique edges and 4 events in [1.0, 9.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([4])
	t		<class 'torch.Tensor'> -> torch.Size([4])
	src		<class 'torch.Tensor'> -> torch.Size([4])

5
4


/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 't', 'src'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [9]:
g = t.to_static_graph()
print(g)

Graph with 9 nodes and 20 edges

Graph attributes
	num_nodes		<class 'int'>



In [10]:
g = t.to_static_graph((1, 10))
print(g)

Graph with 5 nodes and 4 edges

Graph attributes
	num_nodes		<class 'int'>



In [14]:
r = pp.algorithms.RollingTimeWindow(t, 10, 10, return_window=True)
for g, w in r:
    print('Time window ', w)
    print(g)
    print(g.data.edge_index)
    print('---')

Time window  (1.0, 11.0)
Graph with 5 nodes and 4 edges

Graph attributes
	num_nodes		<class 'int'>

tensor([[0, 1, 2, 2],
        [1, 2, 3, 4]])
---
Time window  (11.0, 21.0)
Graph with 7 nodes and 3 edges

Graph attributes
	num_nodes		<class 'int'>

tensor([[2, 5, 0],
        [5, 0, 6]])
---
Time window  (21.0, 31.0)
Graph with 8 nodes and 6 edges

Graph attributes
	num_nodes		<class 'int'>

tensor([[1, 0, 7, 2, 6, 0],
        [5, 6, 5, 5, 7, 2]])
---
Time window  (31.0, 41.0)
Graph with 8 nodes and 3 edges

Graph attributes
	num_nodes		<class 'int'>

tensor([[0, 2, 5],
        [1, 7, 7]])
---
Time window  (41.0, 51.0)
Graph with 9 nodes and 4 edges

Graph attributes
	num_nodes		<class 'int'>

tensor([[1, 8, 2, 7],
        [8, 1, 8, 8]])
---


## Temporal Graphs

Let's start with a simple temporal graph with four nodes `a`,`b`,`c`,`d` and seven timestamped edges `(b,c;2)`,`(a,b;1)`,`(c,d;3)`,`(d,a;4)`,`(b,d;2)`, `(d,a;6)`,`(a,b;7)`. 

The following code generates this temporal graph from the given edge list.

In [15]:
g = pp.TemporalGraph.from_edge_list([['b', 'c', 2],['a', 'b', 1], ['c', 'd', 3], ['d', 'a', 4], ['b', 'd', 2], ['d', 'a', 6], ['a', 'b', 7]])
print(g)

Temporal Graph with 4 nodes, 5 unique edges and 7 events in [1.0, 7.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([7])
	t		<class 'torch.Tensor'> -> torch.Size([7])
	src		<class 'torch.Tensor'> -> torch.Size([7])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 't', 'src'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


We can visualize a temporal graph by using the pathpyG plot function.

In [ ]:
pp.plot(g, edge_color='lightgray')

Consistent with `pyG` the sources, destinations and timestamps are stored as a `pyG TemporalData` object, which we can access in the following way.



In [ ]:
g.data

In [ ]:
print(g.data.t)

With the generator functions `edges` and `temporal_edges` we can iterate through the (temporal) edges of this graph.

In [ ]:
for v, w in g.edges:
    print(v, w)

In [ ]:
for v, w, t in g.temporal_edges:
    print(v, w, t)

## Extracting Causal Topologies via Node-Time Event DAGs

We are often interested in the time respecting paths of a temporal graph.

A time respecting path is defined as a sequence of nodes $v_0,...,v_l$ where the corresponding edges occur in the right time ordering and with a maximum time difference of $\delta\in \N$. 

In order to extract those paths out of a temporal graph, we have to construct a time-unfolded directed acyclic graph (DAG) that represents the network and captures all causal structures.

The nodes of a DAG are node-time-events of the temporal graph, i.e a node `a-t` represents the node `a` at time `t`. 

Two nodes `a-t` and `b-t'` are connected (with exceptions, see code ) if `a-t` directly influences `b-t'` 


In [ ]:
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
print(dag)

print(dag.mapping)
print(dag.data.edge_index)

pp.plot(dag, edge_color='lightgray')

With the following code, we can extract DAGs with only one single root node, which is not influenced by any other node-time event in the temporal graph.

In [ ]:
x = pp.algorithms.extract_causal_trees(dag)
print(x)

## Higher-Order De Bruijn Graph Models for Causal Paths

With the DAG, we can now extract the time-respecting paths in our temporal graph.

In [ ]:
paths = pp.PathData.from_temporal_dag(dag)
print(paths.paths[0])
print(paths.paths[1])
print(g.mapping)

The path data allows us to construct a Higher-Order De Bruijn Graph model belong to our temporal network.

In [ ]:
g2 = pp.HigherOrderGraph(paths, order=2, node_ids=g.mapping.node_ids)
pp.plot(g2)

In a nutshell, the following shows the steps that are needed to construct a Higher Order Graph. Here we construct our temporal graph from a `pyG TemporalData` by providing the sources, distances and timestamps.

In [ ]:
# Create temporal network
d = TemporalData(src=[0,2,1,2,0,2,1,2], dst=[2,3,2,4,2,3,2,4], t=[1,2,3,4,5,6,7,8])
g = pp.TemporalGraph.from_pyg_data(d, node_ids=['a', 'b', 'c', 'd', 'e'])

# Create event DAG and extract path data
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
paths = pp.PathData.from_temporal_dag(dag)

# Create Higher Order Graph
g2 = pp.HigherOrderGraph(paths, order=2, node_ids=g.mapping.node_ids)
pp.plot(g2)

We can also skip the step of creating an event DAG by just using the following code.

In [ ]:
g2 = pp.HigherOrderGraph.from_temporal_graph(g, delta=1, order=2)
pp.plot(g2)